## Prediction of women's shoe price

Background: This dataset has about 10,000 records of woman's shoes and its prices.
    
Data Source: https://data.world/datafiniti/womens-shoe-prices

Objective: To predict the women's shoe price from the brand, model and other specifics
    
Performance metric for machine learning model: R-squared

In [369]:
import numpy as np
import pandas as pd
import sqlite3 
from sklearn.model_selection import train_test_split

## Reading data from CSV file to SQL database

In [370]:
file1 = pd.read_csv(r"7003_1.csv")
file2 = pd.read_csv(r"Datafiniti_Womens_Shoes_Jun19.csv")
file3 = pd.read_csv(r"Datafiniti_Womens_Shoes.csv")

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (25,36,39,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [371]:
file1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19045 entries, 0 to 19044
Data columns (total 47 columns):
id                     19045 non-null object
asins                  2208 non-null object
brand                  18412 non-null object
categories             19045 non-null object
colors                 11889 non-null object
count                  0 non-null float64
dateAdded              19045 non-null object
dateUpdated            19045 non-null object
descriptions           10780 non-null object
dimension              2418 non-null object
ean                    9816 non-null float64
features               14108 non-null object
flavors                0 non-null float64
imageURLs              17840 non-null object
isbn                   0 non-null float64
keys                   19045 non-null object
manufacturer           8656 non-null object
manufacturerNumber     15903 non-null object
merchants              13688 non-null object
name                   19045 non-null object
pri

In [372]:
file2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 32 columns):
id                     10000 non-null object
dateAdded              10000 non-null object
dateUpdated            10000 non-null object
asins                  5 non-null object
brand                  10000 non-null object
categories             10000 non-null object
primaryCategories      10000 non-null object
colors                 10000 non-null object
dimension              465 non-null object
ean / upc              10000 non-null int64
imageURLs              2643 non-null object
keys                   10000 non-null object
manufacturer           1362 non-null object
manufacturerNumber     1431 non-null object
name                   10000 non-null object
prices.amountMax       10000 non-null float64
prices.amountMin       10000 non-null float64
prices.availability    10000 non-null bool
prices.color           10000 non-null object
prices.condition       0 non-null float64
price

In [373]:
file3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 34 columns):
id                     10000 non-null object
dateAdded              10000 non-null object
dateUpdated            10000 non-null object
asins                  3 non-null object
brand                  10000 non-null object
categories             10000 non-null object
primaryCategories      10000 non-null object
colors                 2631 non-null object
dimension              117 non-null object
ean                    671 non-null float64
imageURLs              10000 non-null object
keys                   10000 non-null object
manufacturer           527 non-null object
manufacturerNumber     2482 non-null object
name                   10000 non-null object
prices.amountMax       10000 non-null float64
prices.amountMin       10000 non-null float64
prices.availability    434 non-null object
prices.color           10000 non-null object
prices.condition       438 non-null object
price

In [374]:
file1_cols = [i for i in file1.columns]
file2_cols = [i for i in file2.columns]
file3_cols = [i for i in file3.columns]

In [375]:
print("Total number of columns in File 1: ", len(file1_cols))
print("Total number of columns in File 2: ", len(file2_cols))
print("Total number of columns in File 3: ", len(file3_cols))

Total number of columns in File 1:  47
Total number of columns in File 2:  32
Total number of columns in File 3:  34


File 3 is updated file of file 2. We will use file1 and file 2 for our analysis

In [376]:
file3.sample(5).loc[:,'prices.color':]

,prices.color,prices.condition,prices.currency,prices.dateAdded,prices.dateSeen,prices.isSale,prices.merchant,prices.offer,prices.returnPolicy,prices.shipping,prices.size,prices.sourceURLs,sizes,sourceURLs,upc,weight
4608,Black Patent,NaN,USD,2018-01-29T04:30:01Z,"2018-01-18T13:56:00.000Z,2018-01-05T12:08:00.0...",False,NaN,NaN,NaN,NaN,6.5 MED,https://www.kohls.com/product/prd-2644613/life...,"5 MED,5.5 MED,6 MED,6.5 MED,7 MED,7.5 MED,8 ME...",https://www.kohls.com/product/prd-2644613/life...,"727684713605,727681944729,727686693110,7276859...",NaN
7532,Nude,NaN,USD,2018-01-11T01:19:24Z,"2018-01-05T12:38:00.000Z,2018-01-09T14:32:00.0...",False,NaN,NaN,NaN,NaN,12,https://www.kohls.com/product/prd-2904669/jour...,"5.5,6,6.5,7,7.5,8,8.5,9,10,11,12",https://www.kohls.com/product/prd-2904669/jour...,"870251655843,870251656178,870251655959,8702516...",NaN
2804,Navy,NaN,USD,2018-02-01T07:36:18Z,"2018-01-16T12:04:00.000Z,2018-01-18T13:58:00.0...",False,NaN,NaN,NaN,NaN,6.5 MED,https://www.kohls.com/product/prd-2671544/life...,"5 MED,5.5 MED,6 MED,6.5 MED,7 MED,7.5 MED,8 ME...",https://www.kohls.com/product/prd-2671544/life...,"727679302074,727679301855,727679301190,7276858...",NaN
4692,Brown,NaN,USD,2018-01-02T08:00:47Z,"2017-12-24T14:09:00.000Z,2017-12-20T12:31:00.0...",False,NaN,NaN,NaN,NaN,8 MED,https://www.kohls.com/product/prd-2706606/a2-b...,"5.5 MED,6 MED,6.5 MED,7 MED,7.5 MED,8 MED,8.5 ...",https://www.kohls.com/product/prd-2706606/a2-b...,"737280070354,737280070361,737280070378,7372800...",NaN
9650,Gray/Sky,new,USD,NaN,"2018-04-09T23:00:00Z,2018-04-05T08:00:00Z,2018...",True,Backcountry.com,55%,NaN,Free 2-Day shipping on orders over $50,37,https://www.backcountry.com/scarpa-proton-gtx-...,"39.0,37.5,38.5,40.0,41.0,40.5,37.0,38.0",https://www.backcountry.com/scarpa-proton-gtx-...,NaN,10.6 oz


Let us explore file 3 first as it has information more specific to women's shoes

Create a SQLlite database and create a table named shoes that contains the contents from file3

In [381]:
# create connection to database
connection = sqlite3.connect("womensshoes.db") 
# cursor  
crsr = connection.cursor() 

file3.to_sql('shoes', connection, if_exists='replace', index = False) 

In [382]:
df = pd.read_sql_query("select * from shoes;", connection)
df.columns

Index(['id', 'dateAdded', 'dateUpdated', 'asins', 'brand', 'categories',
       'primaryCategories', 'colors', 'dimension', 'ean', 'imageURLs', 'keys',
       'manufacturer', 'manufacturerNumber', 'name', 'prices.amountMax',
       'prices.amountMin', 'prices.availability', 'prices.color',
       'prices.condition', 'prices.currency', 'prices.dateAdded',
       'prices.dateSeen', 'prices.isSale', 'prices.merchant', 'prices.offer',
       'prices.returnPolicy', 'prices.shipping', 'prices.size',
       'prices.sourceURLs', 'sizes', 'sourceURLs', 'upc', 'weight'],
      dtype='object')

The dataset contains maximum and minimum price for shoes. Let us take the average of these prices and use it as target feature

In [385]:
df['average_price']= (df['prices.amountMax'] + df['prices.amountMin'])/2
df.head(2)

,id,dateAdded,dateUpdated,asins,brand,categories,primaryCategories,colors,dimension,ean,...,prices.offer,prices.returnPolicy,prices.shipping,prices.size,prices.sourceURLs,sizes,sourceURLs,upc,weight,average_price
0,AVpfEf_hLJeJML431ueH,2015-05-04T12:13:08Z,2018-01-29T04:38:43Z,None,Naturalizer,"Clothing,Shoes,Women's Shoes,All Women's Shoes...",Shoes,"Silver,Cream Watercolor Floral",None,NaN,...,None,None,None,S,https://www.overstock.com/Clothing-Shoes/Women...,"6W,9W,7.5W,12W,8.5M,9N,9M,9.5M,10.5M,10W,8.5W,...",https://www.walmart.com/ip/Naturalizer-Danya-W...,017136472311,None,55.990
1,AVpi74XfLJeJML43qZAc,2017-01-27T01:23:39Z,2018-01-03T05:21:54Z,None,MUK LUKS,"Clothing,Shoes,Women's Shoes,Women's Casual Sh...",Shoes,Grey,None,3.397705e+10,...,None,None,Standard,6,https://www.walmart.com/ip/MUK-LUKS-Womens-Jan...,"10,7,6,9,8",https://www.walmart.com/ip/MUK-LUKS-Womens-Jan...,033977045743,None,41.125


## Split data into train and test 

In [386]:
X = df[df.columns[:-1]]
y = df.average_price

In [387]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=.7, test_size=0.3, random_state = 42)
print(X_train.shape)

(7000, 34)


In [388]:
X_train_backup = X_train.copy()

## EDA

In [389]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 9069 to 7270
Data columns (total 34 columns):
id                     7000 non-null object
dateAdded              7000 non-null object
dateUpdated            7000 non-null object
asins                  2 non-null object
brand                  7000 non-null object
categories             7000 non-null object
primaryCategories      7000 non-null object
colors                 1874 non-null object
dimension              81 non-null object
ean                    475 non-null float64
imageURLs              7000 non-null object
keys                   7000 non-null object
manufacturer           367 non-null object
manufacturerNumber     1762 non-null object
name                   7000 non-null object
prices.amountMax       7000 non-null float64
prices.amountMin       7000 non-null float64
prices.availability    319 non-null object
prices.color           7000 non-null object
prices.condition       321 non-null object
prices.currency 

Observation:
    - Most of the features (>85%) are object or categorical type
    - Only 4 numerical features
    - Feature prices.returnPolicy does not contain any information and it can be removed
    - Feature asins has only 2 non-null values. This feature can also be removed
    - Date columns are present which can be converted to pandas timestamp type

In [390]:
# Features to remove
features_remove = {'prices.returnPolicy', 'asins'}

In [391]:
X_train.describe()

,ean,prices.amountMax,prices.amountMin,prices.isSale
count,4.750000e+02,7000.000000,7000.000000,7000.000000
mean,7.729304e+11,69.321727,51.071033,0.018286
std,1.854372e+11,19.423304,20.986210,0.133992
min,3.397705e+10,5.870000,4.960000,0.000000
25%,7.276810e+11,59.990000,37.490000,0.000000
50%,8.701920e+11,64.990000,49.990000,0.000000
75%,8.860660e+11,79.990000,59.990000,0.000000
max,8.898850e+11,359.950000,359.950000,1.000000


In [392]:
### Looking only at non-null features

non_null_df = X_train.loc[:,X_train.columns[X_train.isnull().sum() == 0]]
non_null_df.describe(include = 'all')

,id,dateAdded,dateUpdated,brand,categories,primaryCategories,imageURLs,keys,name,prices.amountMax,prices.amountMin,prices.color,prices.currency,prices.dateSeen,prices.isSale,prices.size,prices.sourceURLs,sizes,sourceURLs
count,7000,7000,7000,7000,7000,7000,7000,7000,7000,7000.000000,7000.000000,7000,7000,7000,7000.000000,7000,7000,7000,7000
unique,611,416,462,119,199,2,611,611,610,NaN,NaN,497,1,3085,NaN,79,694,314,611
top,AV46uYe4glJLPUi8VW57,2017-11-10T19:33:19Z,2018-02-01T07:36:18Z,journee collection,"Womens,Shoes,Boots,Journee Collection",Shoes,https://media.kohlsimg.com/is/image/kohls/2734...,"886548735839,883212328021,886548745760,8844972...",Nike Flex Experience 6 Women's Running Shoes,NaN,NaN,Black,USD,"2018-01-16T11:37:00.000Z,2018-01-17T12:17:00.0...",NaN,6,https://www.kohls.com/product/prd-2734966/nike...,"5 MED,5.5 MED,6 MED,6.5 MED,7 MED,7.5 MED,8 ME...",https://www.kohls.com/product/prd-2734966/nike...
freq,131,181,165,1535,595,6580,131,131,131,NaN,NaN,1399,7000,33,NaN,312,131,513,131
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.321727,51.071033,NaN,NaN,NaN,0.018286,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.423304,20.986210,NaN,NaN,NaN,0.133992,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.870000,4.960000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.990000,37.490000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.990000,49.990000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.990000,59.990000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN


- There are only 2 unique values for the feature primaryCategories.
- All shoe price is in currency USD. The feature prices.currency can be removed

In [393]:
#What are the two unique values of primaryCategories?
non_null_df.primaryCategories.unique()

array(['Shoes,Shoes', 'Shoes'], dtype=object)

PrimaryCategories feature contains - Shoes --> zero variance - this feature will be removed


In [25]:
#What are categories and how is it different from primaryCategories?
print(non_null_df.categories.unique())
#non_null_df[['brand', 'categories']]
non_null_df.groupby('brand')['categories'].nunique().sort_values(ascending = False)

['Womens,Shoes,Boots,Journee Collection'
 'Womens,Shoes,Boots,A2 by Aerosoles'
 'Womens,Shoes,Athletic Shoes,Sneakers,New Balance'
 "Women's Footwear,Women's Running Shoes,Women's Trail Running Shoes"
 "Womens,Shoes,Boots,Dr. Scholl's"
 "Women's Footwear,Women's Casual Boots & Shoes,Women's Casual Shoes"
 "Womens,Shoes,Boots,Journee Collection,Clothing,Women's Shoes,Women's Boots,Heels"
 'Womens,Shoes,Clogs,Mules,Croft,Barrow'
 "Womens,Shoes,Athletic Shoes,Sneakers,Nike,Clothing,Women's Shoes,Women's Casual Shoes,Women's Athletic Shoes"
 'Womens,Shoes,Loafers,NaturalSoul by naturalizer'
 'Womens,Shoes,Loafers,A2 by Aerosoles'
 "Womens,Shoes,Sandals,Easy Street,Clothing,Women's Shoes,All Women's Shoes"
 'Womens,Shoes,Flats,Journee Collection'
 'Womens,Shoes,Pumps,Heels,Style Charles by Charles David'
 'Womens,Shoes,Athletic Shoes,Sneakers,Ryka' 'Womens,Shoes,Flats,Apt. 9'
 "Womens,Shoes,Boots,Candie's"
 'Womens,Shoes,Clogs,Mules,Style Charles by Charles David'
 'Shoes,Boots,Koolaburra b

brand
journee collection      10
easy street             10
lifestride               8
a2 by aerosoles          8
SKECHERS                 8
                        ..
Ryka                     1
kisses by 2 lips too     1
keds                     1
k-swiss                  1
vans                     1
Name: categories, Length: 128, dtype: int64

- "Categories" features has information about the type of shoes - for example, athletic shoes or sandals. In some records, it just contains the brand name. This feature togther with brand name can be used to obtain the different types of footwaer sold by a company. 

- Brand "journee collection" has the highest number of different categories. The different categories are not straightforward but contained within the string:

In [394]:
non_null_df[non_null_df.brand == 'journee collection'].categories.unique()

array(['Womens,Shoes,Flats,Journee Collection',
       'Womens,Shoes,Boots,Journee Collection',
       'Womens,Shoes,Pumps,Heels,Journee Collection',
       "Womens,Shoes,Pumps,Heels,Journee Collection,Clothing,Women's Shoes,Women's Dress Shoes,All Women's Shoes",
       'Womens,Shoes,Clogs,Mules,Journee Collection',
       'Womens,Shoes,Sandals,Journee Collection',
       "Womens,Shoes,Boots,Journee Collection,Clothing,Women's Shoes,All Women's Shoes",
       "Womens,Shoes,Pumps,Heels,Journee Collection,Clothing,Women's Shoes,All Women's Shoes,Women's Boots",
       'Womens,Shoes,Loafers,Journee Collection',
       "Womens,Shoes,Boots,Journee Collection,Clothing,Women's Shoes,Women's Boots,Heels"],
      dtype=object)

Let's see how these information are related to name of the shoe.

In [395]:
non_null_df[non_null_df.brand == 'journee collection'][['name','categories']]

,name,categories
7738,Journee Collection Marlee Women's Pointed Flats,"Womens,Shoes,Flats,Journee Collection"
6461,Journee Collection Ayla Women's High Heel Ankl...,"Womens,Shoes,Boots,Journee Collection"
6096,Journee Collection Cortni Women's Pointed-Toe ...,"Womens,Shoes,Flats,Journee Collection"
6030,Journee Collection Elsa Women's Mary Jane Heels,"Womens,Shoes,Pumps,Heels,Journee Collection"
5898,Journee Collection Wacy Women's Wedges,"Womens,Shoes,Pumps,Heels,Journee Collection,Cl..."
...,...,...
6396,Journee Collection Lara Women's Ankle Boots,"Womens,Shoes,Boots,Journee Collection"
6420,Journee Collection Harlo Women's Mary Jane Heels,"Womens,Shoes,Pumps,Heels,Journee Collection"
6949,Journee Collection Telora Women's Dress Wedges,"Womens,Shoes,Pumps,Heels,Journee Collection"
6265,Journee Collection Piett Women's High Heels,"Womens,Shoes,Pumps,Heels,Journee Collection"


'name' feature has the type/model of shoe. Categories does not provide any new information and thus categories can be removed.

In [396]:
## Features to be removed updated
features_remove.update(['primaryCategories', 'prices.currency', 'categories'])

In [397]:
features_remove

{'asins',
 'categories',
 'prices.currency',
 'prices.returnPolicy',
 'primaryCategories'}

### Date Features

From datainfiniti product schema, 

**dateAdded:** The date this product was first added to datainfiniti's product database. <br>
**dateUpdated:** The most recent date this product was updated or seen by our system.<br>
**prices.dateAdded:** No information<br>
**prices.dateSeen:** A list of dates when this price was seen.<br>

For predicting average price of shoes, these data columns are not necessary. But we can explore this to understand the time frame during which these price information was collected.


In [398]:
## Let us explore the date features
X_train[['dateAdded', 'dateUpdated', 'prices.dateAdded', 'prices.dateSeen', 'prices.amountMax',
       'prices.amountMin']]

,dateAdded,dateUpdated,prices.dateAdded,prices.dateSeen,prices.amountMax,prices.amountMin
9069,2017-11-05T03:20:35Z,2018-01-11T01:19:21Z,2018-01-11T01:19:21Z,"2018-01-05T12:36:00.000Z,2018-01-09T13:31:00.0...",129.99,129.99
2603,2017-11-24T12:25:07Z,2018-02-01T07:35:56Z,2017-11-24T12:25:07Z,2017-11-12T09:36:00.000Z,69.99,39.99
7738,2017-11-20T21:09:48Z,2018-02-01T07:35:44Z,2018-01-02T17:29:14Z,"2017-12-19T11:52:00.000Z,2017-12-15T13:59:00.0...",59.99,42.49
1579,2017-11-17T15:56:59Z,2018-01-21T03:17:22Z,2017-11-25T22:28:55Z,"2017-11-02T08:25:00.000Z,2017-11-03T09:30:00.000Z",59.99,27.99
5058,2017-11-10T19:33:26Z,2018-01-29T04:27:28Z,2017-12-31T11:44:39Z,"2017-12-25T16:57:00.000Z,2017-12-04T13:58:00.0...",89.99,52.49
...,...,...,...,...,...,...
5734,2015-08-14T18:09:12Z,2018-02-01T07:33:04Z,2018-02-01T07:33:04Z,"2018-01-16T08:27:00.000Z,2018-01-17T14:44:00.0...",79.99,79.99
5191,2017-11-10T19:33:46Z,2018-01-21T02:19:53Z,2017-12-31T04:46:33Z,"2017-12-04T15:29:00.000Z,2017-12-03T06:44:00.0...",89.00,52.49
5390,2017-11-10T19:33:19Z,2018-02-13T21:53:04Z,2018-01-11T00:03:43Z,"2018-01-05T11:40:00.000Z,2018-01-09T14:10:00.0...",119.00,119.00
860,2017-09-12T03:29:25Z,2018-02-01T07:34:14Z,2018-02-01T07:34:14Z,"2018-01-16T09:58:00.000Z,2018-01-15T11:52:00.0...",54.99,54.99


In [399]:
X_train[['dateAdded', 'dateUpdated', 'prices.dateAdded']] = X_train[['dateAdded', 'dateUpdated', 'prices.dateAdded']].apply(pd.to_datetime)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [400]:
X_train[['dateAdded', 'dateUpdated', 'prices.dateAdded']].describe()

,dateAdded,dateUpdated,prices.dateAdded
count,7000,7000,6449
unique,416,462,1743
top,2017-11-10 19:33:19+00:00,2018-02-01 07:36:18+00:00,2018-02-01 07:32:58+00:00
freq,181,165,58
first,2015-07-20 22:46:58+00:00,2018-01-01 05:51:15+00:00,2015-11-30 01:47:56+00:00
last,2018-05-21 13:17:40+00:00,2018-10-23 21:26:11+00:00,2018-02-14 20:58:11+00:00


The data in this dataset were collected from May 2015  - October 2018. 

In [401]:
features_remove.update(['dateAdded', 'dateUpdated', 'prices.dateAdded', 'prices.dateSeen', 'prices.amountMax',
       'prices.amountMin'])

In [402]:
X_train[['colors', 'prices.color']]

,colors,prices.color
9069,None,Black Pink
2603,None,Gray
7738,None,Black
1579,None,Black
5058,None,Brown
...,...,...
5734,"Gold Metallic,Black,Medium Brown,Bordo,Peach,M...",Medium Brown
5191,None,Navy
5390,None,Slate
860,Blue,Navy


Colors field indicates all the colors that are available for a particular shoe. The price of a shoe can vary with color. The specific color is specified by prices.color and therefore colors can be removed.

In [403]:
features_remove.update(['colors'])

Let's look at the different URLs. We are not going to use image for our predition and thus, image url can de removed. 

In [406]:
# Are sourceurl and prices.sourceurls the same?
url = X_train[X_train.columns[X_train.columns.str.contains("URL")]]
url

,imageURLs,prices.sourceURLs,sourceURLs
9069,https://media.kohlsimg.com/is/image/kohls/2480...,https://www.kohls.com/product/prd-2480285/asic...,https://www.kohls.com/product/prd-2480285/asic...
2603,https://media.kohlsimg.com/is/image/kohls/2908...,https://www.kohls.com/product/prd-2908377/so-w...,https://www.kohls.com/product/prd-2908377/so-w...
7738,https://media.kohlsimg.com/is/image/kohls/2948...,https://www.kohls.com/product/prd-2948334/jour...,https://www.kohls.com/product/prd-2948334/jour...
1579,https://media.kohlsimg.com/is/image/kohls/2532...,https://www.kohls.com/product/prd-2532138/crof...,https://www.kohls.com/product/prd-2532138/crof...
5058,https://media.kohlsimg.com/is/image/kohls/2923...,https://www.kohls.com/product/prd-2923522/a2-b...,https://www.kohls.com/product/prd-2923522/a2-b...
...,...,...,...
5734,https://media.kohlsimg.com/is/image/kohls/2769...,https://www.kohls.com/product/prd-2769027/spri...,https://www.kohls.com/product/prd-2769027/spri...
5191,https://media.kohlsimg.com/is/image/kohls/2811...,https://www.kohls.com/product/prd-2811198/styl...,https://www.kohls.com/product/prd-2811198/styl...
5390,https://media.kohlsimg.com/is/image/kohls/2584...,https://www.kohls.com/product/prd-2584698/styl...,https://www.kohls.com/product/prd-2584698/styl...
860,https://media.kohlsimg.com/is/image/kohls/2966...,https://www.kohls.com/product/prd-2966724/adid...,https://www.kohls.com/product/prd-2966724/adid...


In [407]:
print("sourceURLs and prices.sourceURLs are the same:", (url.sourceURLs == url['prices.sourceURLs']).all())

sourceURLs and prices.sourceURLs are the same: False


In [408]:

X_train[['brand', 'manufacturer', 'name', 'sourceURLs']]

,brand,manufacturer,name,sourceURLs
9069,asics,None,ASICS GEL Quantum 180 2 Women's Running Shoes,https://www.kohls.com/product/prd-2480285/asic...
2603,so,None,SO Follow Women's Ankle Boots,https://www.kohls.com/product/prd-2908377/so-w...
7738,journee collection,None,Journee Collection Marlee Women's Pointed Flats,https://www.kohls.com/product/prd-2948334/jour...
1579,croft barrow,None,Croft Barrow Women's Ortholite Quilted Ballet ...,https://www.kohls.com/product/prd-2532138/crof...
5058,a2 by aerosoles,None,A2 by Aerosoles Gravity Women's Ankle Boots,https://www.kohls.com/product/prd-2923522/a2-b...
...,...,...,...,...
5734,spring step,Spring Footwear,Spring Step Happy Women's Clogs,https://www.kohls.com/product/prd-2769027/spri...
5191,style charles by charles david,None,Style Charles by Charles David Laura Women's W...,https://www.kohls.com/product/prd-2811198/styl...
5390,style charles by charles david,None,Style Charles by Charles David Groove Women's ...,https://www.kohls.com/product/prd-2584698/styl...
860,adidas,None,adidas NEO Courtset Women's Suede Sneakers,https://www.kohls.com/product/prd-2966724/adid...


In [409]:
# A list of merchants selling this product. These are typically third-party merchants found on
#e-commerce websites.

X_train[~X_train['prices.merchant'].isnull()]
X_train['prices.merchant'].unique()

array([None, 'Backcountry.com', 'Walmart.com',
       'DAILYWEAR  SPORTSWEAR CORP.', 'AmazingBasics', 'Slippers Dot Com',
       'Overstock.com', 'UnbeatableSale', 'Tasharina Corp', 'Shoebuy.com'],
      dtype=object)

In [410]:
X_train[X_train['prices.merchant'] ==  'DAILYWEAR  SPORTSWEAR CORP.']

,id,dateAdded,dateUpdated,asins,brand,categories,primaryCategories,colors,dimension,ean,...,prices.merchant,prices.offer,prices.returnPolicy,prices.shipping,prices.size,prices.sourceURLs,sizes,sourceURLs,upc,weight
142,AVph34uzilAPnD_x-STz,2017-01-23 05:30:38+00:00,2018-01-30 06:16:48+00:00,None,Victoria K.,"Clothing,Shoes,Women's Shoes,All Women's Shoes",Shoes,"Black,Red",None,6.635160e+11,...,DAILYWEAR SPORTSWEAR CORP.,None,None,Value,6,https://www.walmart.com/ip/Victoria-K-Women-s-...,"10,7,6,9,8",https://www.walmart.com/ip/Victoria-K-Women-s-...,663516000000,None


Are price.sourceurl and source url the same?

In [411]:
X_train.sourceURLs == X_train['prices.sourceURLs']
psl = X_train['prices.sourceURLs'].str.extract(r'(.+?(?=.com))')
surl = X_train['sourceURLs'].str.extract(r'(.+?(?=.com))')
pd.concat([surl,psl], axis =1)[~(surl[0] == psl[0])]

,0,0
8966,http://www.walmart,https://www.kohls
122,https://www.walmart,https://www.kohls
3445,https://www.walmart,https://www.kohls
8409,https://www.walmart,https://www.kohls
3438,https://www.walmart,https://www.kohls
...,...,...
3748,https://www.walmart,https://www.kohls
64,https://www.walmart,https://www.kohls
8571,https://www.walmart,https://www.kohls
8529,https://www.walmart,https://www.kohls


**Price source url:** Products sold by walmart.com, third party in e-commerce site <br>
**sourceURLs:** A list of URLs used to generate data for this product. <br>

-- Source url and price source url are not the same. <br>
-- Since we obtain price from price source url, we can use company from that field as merchant if merchant is not found.<br>
-- After the above steps, all the urls can be eliminated.<br>

In [412]:
features_remove.update(['prices.sourceURLs', 'sourceURLs', 'imageURLs' ])

In [413]:
### Imputing null values in prices.merchant 

merchant_imputation = X_train[['prices.merchant', 'prices.sourceURLs']]
merchant_imputation['prices.sourceURLs'] = merchant_imputation['prices.sourceURLs'].str.extract(r'((?<=www.).+?(?=/))')
merchant_imputation['prices.merchant'] = merchant_imputation['prices.merchant'].str.lower()
#(merchant_imputation[~merchant_imputation['prices.merchant'].isnull()]['prices.merchant'] == merchant_imputation[~merchant_imputation['prices.merchant'].isnull()]['prices.sourceURLs']).all()
print("Unique values in merchant:", merchant_imputation['prices.merchant'].unique())
print("Unique values in source url:", merchant_imputation['prices.sourceURLs'].unique())

Unique values in merchant: [None 'backcountry.com' 'walmart.com' 'dailywear  sportswear corp.'
 'amazingbasics' 'slippers dot com' 'overstock.com' 'unbeatablesale'
 'tasharina corp' 'shoebuy.com']
Unique values in source url: ['kohls.com' 'backcountry.com' 'walmart.com' 'evo.com' 'overstock.com']


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


- Merchant contains the name of the third part seller from the e-commerce site if there is one. Company name from price source url contains only the e-commerce company name.

- We will fill all null values in merchant with e-commerce company name from source url. 

In [414]:
X_train['prices.merchant'] = X_train[['prices.merchant', 'prices.sourceURLs']].apply(lambda x: re.findall(r'((?<=www.).+?(?=/))', x['prices.sourceURLs'])[0] if pd.isnull(x['prices.merchant']) else x['prices.merchant'], axis = 1)
(X_train['prices.merchant'].isnull()).all()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


False

In [415]:
### Understanding features - manufacture and manufacturer and how it is related to name and brand
# Manufacturer: The manufacturer of this product.
## Manufacturernumber: The manufacturer or model number of this product.
X_train[['manufacturer', 'manufacturerNumber', 'name', 'brand']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 9069 to 7270
Data columns (total 4 columns):
manufacturer          367 non-null object
manufacturerNumber    1762 non-null object
name                  7000 non-null object
brand                 7000 non-null object
dtypes: object(4)
memory usage: 273.4+ KB


In [416]:
manufacture = X_train[['manufacturer', 'manufacturerNumber', 'name', 'brand']][~X_train.manufacturer.isnull()]
manufacture.groupby('manufacturer')['brand'].unique()

manufacturer
ASICS                                              [asics]
Brinley Co                                   [Brinley Co.]
Brinley Co.              [journee collection, Brinley Co.]
Dr. Scholl's                                 [dr. scholls]
ELLIE SHOES                                  [ELLIE SHOES]
Eastland Shoe                                   [eastland]
Easy Street                                  [easy street]
Fila                                                [fila]
Journee Collection                           [Brinley Co.]
Lugz                                                [Lugz]
Muk Luks                                        [MUK LUKS]
Reliable of Milwaukee                           [MUK LUKS]
Reliable of Milwuakee                           [MUK LUKS]
S-1944-red-6                                 [Victoria K.]
Sloggers                                        [Sloggers]
Soft Ones                                      [Soft Ones]
Spring Footwear                            

- Manufacturer and brand are the same except for Journee Collection and Brinley Co. On searching for these brands in walmart.com, it is lear that there are some shoes which are titled Brinley Co. and the actual shoes are from journee colelction (https://www.walmart.com/browse/clothing/brinley-co-/journee-collection/5438_3317124_7499317/YnJhbmQ6Sm91cm5lZSBDb2xsZWN0aW9u). Thus, we can remove manufacturer name. 

In [417]:
### looks like manufacturer and brand name are same. Let's check
manufacture_number = X_train[['manufacturer', 'manufacturerNumber', 'name', 'brand']][~X_train.manufacturerNumber.isnull()]
manufacture_number.groupby('manufacturerNumber')['name'].nunique().sort_values(ascending = False)

manufacturerNumber
skx14505bbk        1
AVALON-BLK-060     1
C8149M7900         1
BRK004G            1
BRJ004S            1
                  ..
NEW00DG            1
NEW00D2            1
NELSON02-NUD060    1
MER00IP            1
0015589605-S       1
Name: name, Length: 216, dtype: int64

Manufacturer and name represents the model of the shoe. Every manufacturer number corresponds to the model of the shoe given in 'name' feature. Thus, we can remove one of these features.

In [418]:
## list of brands for which manufacturerNumber is missing
print(X_train[X_train.manufacturerNumber.isnull()]['brand'].unique())
X_train[X_train.brand == 'ryka'][['name', 'manufacturerNumber']]

['asics' 'so' 'journee collection' 'croft barrow' 'a2 by aerosoles'
 'lifestride' 'style charles by charles david' 'new balance' 'candies'
 'seven7' 'skechers' 'nike' 'sonoma goods for life' 'unionbay' 'rampage'
 'ryka' 'simply vera vera wang' 'easy street' 'naturalsoul by naturalizer'
 'keds' 'clarks' 'madden nyc' 'apt. 9' 'adidas' 'Lifestride'
 'dolce by mojo moxy' 'New Balance' 'vans' 'eastland' 'koolaburra by ugg'
 'reef' 'Red Wing' '2 lips too' 'dr. scholls' 'soft style by hush puppies'
 'lc lauren conrad' 'kisses by 2 lips too' 'rocky 4eursole' 'puma'
 'forever collectibles' 'jennifer lopez' 'henry ferrera' 'Keds' 'Laredo'
 'SKECHERS' 'andrew geller' 'under armour' 'spring step' 'sugar' 'qupid'
 'Nike' 'nyla' 'olivia miller' 'lugz' 'Unique Bargains' 'herstar' 'corkys']


,name,manufacturerNumber
3495,Ryka Fierce Women's Walking Shoes,None
4264,Ryka Propel 3D Pro Women's Walking Shoes,None
3405,Ryka Dominion Women's Walking Shoes,E3616M1400
3577,Ryka Dominica Women's Sandals,E3800F1020
3768,Ryka Dash 3 Women's Walking Shoes,None
...,...,...
4433,Ryka Nora Women's Wedge Sandals,None
4389,Ryka Dream Women's Cross-Training Shoes,None
3581,Ryka Dominica Women's Sandals,E3800F1020
3561,Ryka Savannah Women's Sandals,E6490M1002


Since 'name' feature has no null values, we will retain it and remove manufacturerNumber.

In [419]:
features_remove.update(['manufacturer', 'manufacturerNumber'])

### The following features will be removed as they are not relevant for shoe pricing:

- sizes: This feature lists all the sizes available in the market. But we have another feature named prices.size which gives the size for which the shoe is priced 
- upc: These are unique barcode number specific to product. More than one upc for an observation idicates the variation of the product. Let us remove it for now.
- keys: These are keys for data infiniti internal usage and will be removed
-
- All features with >90% of the data is missing will be removed: weight, dimension, shipping, prices. condition, prices.availability
- prices.offer will be retained as it is an important information for pricing

In [420]:
## All features in the dataset which have >90% of missing values. 

rows = X_train.shape[0]
feature_greater90_null = [cols for cols in X_train.columns if (X_train[cols].isnull().sum()/rows) > 0.9]
feature_greater90_null 

['asins',
 'dimension',
 'ean',
 'manufacturer',
 'prices.availability',
 'prices.condition',
 'prices.offer',
 'prices.returnPolicy',
 'prices.shipping',
 'weight']

In [421]:
features_remove.update(feature_greater90_null)
features_remove.remove('prices.offer')
features_remove

{'asins',
 'categories',
 'colors',
 'dateAdded',
 'dateUpdated',
 'dimension',
 'ean',
 'imageURLs',
 'manufacturer',
 'manufacturerNumber',
 'prices.amountMax',
 'prices.amountMin',
 'prices.availability',
 'prices.condition',
 'prices.currency',
 'prices.dateAdded',
 'prices.dateSeen',
 'prices.returnPolicy',
 'prices.shipping',
 'prices.sourceURLs',
 'primaryCategories',
 'sourceURLs',
 'weight'}

In [422]:
features_remove.update(['upc', 'sizes', 'keys'])

In [423]:
X_train.drop(columns = list(features_remove), inplace = True)
X_train

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,id,brand,name,prices.color,prices.isSale,prices.merchant,prices.offer,prices.size
9069,AV-KMgH3YSSHbkXwpN8m,asics,ASICS GEL Quantum 180 2 Women's Running Shoes,Black Pink,0,kohls.com,None,8
2603,AV_t__-fHh53nbDRA9X6,so,SO Follow Women's Ankle Boots,Gray,0,kohls.com,None,9
7738,AV_cO7IHYSSHbkXwq03F,journee collection,Journee Collection Marlee Women's Pointed Flats,Black,0,kohls.com,None,8.5
1579,AV_K_zzHHh53nbDRAB-4,croft barrow,Croft Barrow Women's Ortholite Quilted Ballet ...,Black,0,kohls.com,None,10 WIDE
5058,AV-ndHZmYSSHbkXwpawS,a2 by aerosoles,A2 by Aerosoles Gravity Women's Ankle Boots,Brown,0,kohls.com,None,8 MED
...,...,...,...,...,...,...,...,...
5734,AVpg4tTH1cnluZ0-7irp,spring step,Spring Step Happy Women's Clogs,Medium Brown,0,kohls.com,None,39
5191,AV-ndG1uKZqtpbFMS0SV,style charles by charles david,Style Charles by Charles David Laura Women's W...,Navy,0,kohls.com,None,7.5 MED
5390,AV-nbkV9Hh53nbDR_Uxh,style charles by charles david,Style Charles by Charles David Groove Women's ...,Slate,0,kohls.com,None,10 MED
860,AV50Jgvy-jtxr-f3ILyv,adidas,adidas NEO Courtset Women's Suede Sneakers,Navy,0,kohls.com,None,6


In [424]:
#what is id? Is it a unique value?
print("How many unique ids are there?", X_train.id.nunique())
print("Are there any duplicates? If so how many?", X_train.duplicated().sum())
#X_train.groupby('id')['prices.size'].nunique().sort_values()

How many unique ids are there? 611
Are there any duplicates? If so how many? 1168


After removing the unwanted features, the dataframe finally has 8 columns

In [425]:
X_train_backup.duplicated().sum()

0

In [426]:
file3.duplicated().any()

False